## Notebook

2 DataFrames will be completed by the end of this. One will contain all the bills. The other will contain all the reps.

For each bill do the following:
1. Get the sponsor plus state -> under "sponsors"
    1. Get sponsor party -> from API
    2. Get sponsor seniority -> from API
2. Get the cosponsors -> under "cosponsors"
    1. Get cosponsor party -> from API
3. Pass or fail -> under "status"
4. Date introduced -> under "introduced_at"
5. Date passed -> under "?"
6. Bipartisanship score based off of (co)sponsor party ratios


While going through each bill, iteratively complete representative df
1. Representative
2. Bills introduced
3. Average "bipartisanship score of bills"

### Updates for Rohun:

1. Cleaning not completely done. Need to still create the bipartisanship score for each bill. Use that to create bipartisanship scores for each Rep. 

2. Using the bipartisanship score and seniority, plot all the reps. Bipartisanship scores should be (-1-0) for Dems and (0-1) for GOP

In [1]:
import pandas as pd
import numpy as np
import json

from pandas.io.json import json_normalize

## Functions

In [48]:
#Get sponsor party affiliation
def get_party(sponsor):
    party = reps_with_bills.loc[reps_with_bills.id == sponsor]
    party = party['party'].values[0]
    return(party)    

In [56]:
#Get consponsor party affiliations
def get_cosponsor_party(cosponsors):
    if len(cosponsors) == 0:
        return ((0, 0))
    else:
        parties = []
        for i in range(len(cosponsors)):
            party = reps_with_bills.loc[reps_with_bills.id == cosponsors[i]]['party'].values[0]
            parties.append(party)
        GOP = parties.count('R')
        DEM = parties.count('D')
        return((GOP, DEM))
        

## Read in Reps File

Create a lookup table for the Rep Ids. Table should include Name, Party, Seniority, State, Gender, 

In [10]:
reps_df = pd.read_csv('/Volumes/Rohun Backups/Datasets/Congress/reps/reps_115.csv')
reps_with_bills = reps_df[['id', 'first_name', 'last_name', 'party', 'seniority', 'state', 'gender']]

reps_with_bills['Name'] = reps_with_bills['first_name'] + '_' + reps_with_bills['last_name']

/Users/rohun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [11]:
reps_with_bills.head()

,id,first_name,last_name,party,seniority,state,gender,Name
0,A000374,Ralph,Abraham,R,4,LA,M,Ralph_Abraham
1,A000370,Alma,Adams,D,6,NC,F,Alma_Adams
2,A000055,Robert,Aderholt,R,22,AL,M,Robert_Aderholt
3,A000371,Pete,Aguilar,D,4,CA,M,Pete_Aguilar
4,A000372,Rick,Allen,R,4,GA,M,Rick_Allen


## Read through Bills

In [3]:
#all bills dict
all_bills = {}

In [4]:
for bills in range(0,7400):
    if bills % 200 == 0:
        print(str(bills) + " have been completed")
    try:
        bill_num = 'hr' + str(bills)
        with open('/Volumes/Rohun Backups/Datasets/Congress/congress/data/115/bills/hr/'+ bill_num + '/data.json') as json_file:  
            bill_details = json.load(json_file)

        #get bill id
        bill_id = bill_details['bill_id']

        #get sponsor
        sponsor_id = bill_details['sponsor']['bioguide_id']

        #get number of cosponsors and list of consponsors
        num_cosponsors = len(bill_details['cosponsors'])
        cosponsors = []
        for i in range(num_cosponsors):
            cosponsors.append(bill_details['cosponsors'][i]['bioguide_id'])

        #get bill status
        status = bill_details['status']

        #get introduction date
        intro_date = bill_details['introduced_at']

        #get pass/fail date
        end_date = bill_details['status_at']
        
        #get committee
        num_committees = len(bill_details['committees'])
        committees = []
        for k in range(num_committees):
            committees.append(bill_details['committees'][k]['committee'])


        curr_bill = {'bill_id': bill_id, 'sponsor_id': sponsor_id, 'num_cosponsors': num_cosponsors,
                     'cosponsors': cosponsors, 'committees': committees, 'status': status, 
                     'intro_date': intro_date, 'end_date': end_date}
        all_bills[bills] = curr_bill
    
    except:
        print("Bill Number " + str(bills) + " doesn't exist")

0 have been completed
Bill Number 0 doesn't exist
Bill Number 9 doesn't exist
Bill Number 11 doesn't exist
Bill Number 13 doesn't exist
Bill Number 14 doesn't exist
Bill Number 16 doesn't exist
Bill Number 17 doesn't exist
Bill Number 18 doesn't exist
200 have been completed
400 have been completed
600 have been completed
800 have been completed
1000 have been completed
1200 have been completed
1400 have been completed
1600 have been completed
1800 have been completed
2000 have been completed
2200 have been completed
2400 have been completed
2600 have been completed
2800 have been completed
3000 have been completed
3200 have been completed
3400 have been completed
3600 have been completed
3800 have been completed
4000 have been completed
4200 have been completed
4400 have been completed
4600 have been completed
4800 have been completed
5000 have been completed
5200 have been completed
5400 have been completed
5600 have been completed
5800 have been completed
6000 have been completed
62

In [5]:
final_bill_df = []
for key in all_bills:
    temp = json_normalize(all_bills[key])
    final_bill_df.append(temp)
final_bill_df = pd.concat(final_bill_df, ignore_index=True)

In [6]:
final_bill_df.head()

,bill_id,committees,cosponsors,end_date,intro_date,num_cosponsors,sponsor_id,status
0,hr1-115,[House Ways and Means],"[B001293, B001273, B001260, C001107, H001065, ...",2017-12-22,2017-11-02,24,B000755,ENACTED:SIGNED
1,hr2-115,[House Agriculture],[],2018-12-20,2018-04-12,0,C001062,ENACTED:SIGNED
2,hr3-115,"[Senate Appropriations, Senate Budget, House A...","[A000055, C001053, C001093, D000619, F000450, ...",2018-06-07T21:18:53-04:00,2018-05-09,16,M001165,PASS_OVER:HOUSE
3,hr4-115,"[House Natural Resources, House Natural Resour...","[B001269, C001037, D000191, D000612, G000559, ...",2018-04-27T11:39:30-04:00,2018-04-13,15,S001154,PASS_OVER:HOUSE
4,hr5-115,"[Senate Small Business and Entrepreneurship, S...","[A000375, B001293, B001289, C000266, C001093, ...",2017-01-11T18:46:57-05:00,2017-01-03,25,G000289,PASS_OVER:HOUSE


In [7]:
final_bill_df.shape

(7392, 8)

In [44]:
#get sponsor party
final_bill_df['sponsor_party'] = final_bill_df.sponsor_id.apply(lambda x: get_party(x))

In [59]:
#get dem/rep consponsor counts
final_bill_df['cosponsor_parties_gop_dem'] = final_bill_df.cosponsors.apply(lambda x: get_cosponsor_party(x))

In [60]:
final_bill_df.head()

,bill_id,committees,cosponsors,end_date,intro_date,num_cosponsors,sponsor_id,status,sponsor_party,cosponsor_parties_gop_dem
0,hr1-115,[House Ways and Means],"[B001293, B001273, B001260, C001107, H001065, ...",2017-12-22,2017-11-02,24,B000755,ENACTED:SIGNED,R,"(24, 0)"
1,hr2-115,[House Agriculture],[],2018-12-20,2018-04-12,0,C001062,ENACTED:SIGNED,R,"(0, 0)"
2,hr3-115,"[Senate Appropriations, Senate Budget, House A...","[A000055, C001053, C001093, D000619, F000450, ...",2018-06-07T21:18:53-04:00,2018-05-09,16,M001165,PASS_OVER:HOUSE,R,"(16, 0)"
3,hr4-115,"[House Natural Resources, House Natural Resour...","[B001269, C001037, D000191, D000612, G000559, ...",2018-04-27T11:39:30-04:00,2018-04-13,15,S001154,PASS_OVER:HOUSE,R,"(8, 7)"
4,hr5-115,"[Senate Small Business and Entrepreneurship, S...","[A000375, B001293, B001289, C000266, C001093, ...",2017-01-11T18:46:57-05:00,2017-01-03,25,G000289,PASS_OVER:HOUSE,R,"(24, 1)"


In [61]:
final_bill_df.to_csv('/Volumes/Rohun Backups/Datasets/Congress/cleaned_115_bills.csv')